### Reconstructing the norm groups by graph coloring.

In this notebook we use the greedy graph coloring algorithm from the ```networkx``` package to determine a lower bound on the number of norm groups that were used for the raw-to-decile conversion in ProPublica's dataset. The notebook produces an assignment of people to norm groups, and uses that to compute the cut-points associated to those norm groups. Assign ```True``` to ```split_by_sex``` to split the population in male/female groups before performing the analysis. 

In [160]:
import pandas as pd
import networkx as nx
from itertools import combinations

# This switch first splits the population by male/female before reconstructing the norm groups. 
split_by_sex = True

# Returns whether the order between i and j equals that between k and l. 
def cons(i, k, j, l):
    return (i == j and k == l) or (i < j and k <= l) or (j < i and l <= k)

# Load the fta bins csv.    
df = pd.read_csv("data/slevel.csv")

for score in ['fta', 'grecid', 'vrecid']:
    print("\n=====================")
    print(f"Analyzing {score}.")
    print("=====================")
    
    # Initialize the df. 
    sdf = None

    for s in [0, 1] if split_by_sex else [2]:
        if s in [0, 1]:
            print(f"\nAnalyzing {'male' if s == 1 else 'female'} population.")
            sdf = df[df["male"] == s][[f"{score}_{st}" for st in ['rs', 'ds']] + ["male", "person_id"]]
        elif s == 2:
            sdf = df[[f"{score}_{st}" for st in ['rs', 'ds']] + ["male", "person_id"]]

        # Create the set of vertices. 
        V = list(sdf.index)

        # Lookup dict for score values. This is used to speed up computations.
        d = {i : [sdf.loc[i, f'{score}_rs'], sdf.loc[i, f'{score}_ds']] for i in V}

        # Create the set of edges.
        E = [(i, j) for i, j in combinations(V, 2) if not cons(*d[i], *d[j])]
        
        # Create the networkx Graph object. 
        G = nx.Graph()
        G.add_nodes_from(V)
        G.add_edges_from(E)
        print(f"The graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.\n")

        # Try to find a clique of size 3.
        clique = None
        for c in nx.find_cliques(G):
            if len(c) == 3:
                clique = c
                break

        # Print the score values of the members of the clique, if one was found.
        if clique:
            print(f"Found a clique of size 3:")
            for i in clique:
                print(f"Node {i} (pid = {sdf.loc[i, 'person_id']}) has scores {d[i]}")
            print("This means at least 3 colors are needed to color graph.\n")

        # Compute a graph coloring.
        graph_coloring = nx.greedy_color(G)
        colors = set(graph_coloring.values())
        print(f"Colored the graph using {len(colors)} color(s).")

        # Compute the minimum and maximum raw values per decile, for each color. 
        CV = {c : [v for v in graph_coloring if graph_coloring[v] == c] for c in colors}
        for c in colors:
            print()
            perc = round(100 * len([v for v in CV[c] if sdf.loc[v, "male"] == 1]) / len(CV[c]), 2)          
            print(f"Showing bounds for color {c} (containing {len(CV[c])} nodes, of which {perc}% male):")
            cd = {i : [d[v][0] for v in CV[c] if d[v][1] == i] for i in range(1, 11)}
            for i in range(1, 11):
                if cd[i] != []:
                    print(i, min(cd[i]), max(cd[i]), len(cd[i]))
                else: 
                    print(i, "None with this decile/color combination.")


Analyzing fta.

Analyzing female population.
The graph has 2508 nodes and 1245 edges.

Colored the graph using 2 color(s).

Showing bounds for color 0 (containing 1787 nodes, of which 0.0% male):
1 11.0 16.0 1138
2 17.0 17.0 160
3 18.0 20.0 305
4 21.0 22.0 104
5 24.0 24.0 2
6 25.0 25.0 1
7 26.0 28.0 32
8 29.0 31.0 4
9 32.0 33.0 20
10 34.0 40.0 21

Showing bounds for color 1 (containing 721 nodes, of which 0.0% male):
1 None with this decile/color combination.
2 16.0 16.0 214
3 None with this decile/color combination.
4 20.0 20.0 124
5 22.0 22.0 83
6 23.0 24.0 129
7 25.0 25.0 53
8 27.0 28.0 62
9 29.0 31.0 56
10 None with this decile/color combination.

Analyzing male population.
The graph has 10002 nodes and 39831 edges.

Colored the graph using 2 color(s).

Showing bounds for color 0 (containing 5295 nodes, of which 100.0% male):
1 11.0 16.0 3653
2 17.0 19.0 736
3 20.0 21.0 527
4 22.0 23.0 11
5 24.0 25.0 13
6 26.0 27.0 9
7 28.0 29.0 13
8 30.0 31.0 100
9 32.0 35.0 130
10 36.0 50.0 103
